In [15]:
import sympy as sp
import time
from tabulate import tabulate
import matplotlib.pyplot as plt
import numpy as np
import math

In [16]:
#write units for below variables
values = {
    'F': 1, #m^3/hr
    'CAf': 10, #kgmol/m^3
    'V' :1,#m^3
    'UA': 150, #kcal/degC-hr
    'k0' : 36*1e6, #h-1
    'Tj0' : 298, #K
    '-delH': 6500, #kcal/kgmol
    'rhojCj': 600, #kcal/m^3-degC
    'E' : 12000, #kcal/kgmol
    'Fj' :1.25, #m^3/hr
    'rhoCp': 500, #kcal/m^3-degC
    "Vj" : 0.25, #m^3
    'Tf' : 298, #K
    'R': 1.987 #kcal/kgmol-K
}

In [28]:
F, C_Af, V, UA, k0, Tj0, neg_delH, rhojCj, E, Fj, rhoCp, Vj, Tf, C_a, T, Tj, r, R = sp.symbols('F C_Af V UA k0 T_j0 -delH rho_jC_j E F_j rhoCp V_j T_f C_a T T_j r R', real = True)

In [29]:
r = sp.exp(-E/(R*T))*C_a*k0
r

C_a*k0*exp(-E/(R*T))

In [31]:
f_1 = F*C_Af - F*C_a - r*V
f_1

C_Af*F - C_a*F - C_a*V*k0*exp(-E/(R*T))

In [32]:
f_2 = rhoCp*F*(Tf-T) - UA*(T-Tj) + r*V*neg_delH
f_2

-delH*C_a*V*k0*exp(-E/(R*T)) + F*rhoCp*(-T + T_f) - UA*(T - T_j)

In [34]:
f_3 = rhojCj*Fj*(Tj0-Tj) - UA*(T-Tj)
f_3

F_j*rho_jC_j*(-T_j + T_j0) - UA*(T - T_j)

In [43]:
#Generating a matrix of the equations
f = sp.Matrix([f_1, f_2, f_3])
f

Matrix([
[                         C_Af*F - C_a*F - C_a*V*k0*exp(-E/(R*T))],
[-delH*C_a*V*k0*exp(-E/(R*T)) + F*rhoCp*(-T + T_f) - UA*(T - T_j)],
[                       F_j*rho_jC_j*(-T_j + T_j0) - UA*(T - T_j)]])

In [36]:
#Generating Jacobian matrix
J = sp.Matrix([f_1, f_2, f_3]).jacobian([C_a, T, Tj])
J

Matrix([
[ -F - V*k0*exp(-E/(R*T)),                     -C_a*E*V*k0*exp(-E/(R*T))/(R*T**2),                  0],
[-delH*V*k0*exp(-E/(R*T)), -delH*C_a*E*V*k0*exp(-E/(R*T))/(R*T**2) - F*rhoCp - UA,                 UA],
[                       0,                                                    -UA, -F_j*rho_jC_j + UA]])

In [49]:
f = f.subs([
    (F, values['F']),
    (C_Af, values['CAf']),
    (V, values['V']),
    (UA, values['UA']),
    (k0, values['k0']),
    (Tj0, values['Tj0']),
    (neg_delH, values['-delH']),
    (rhojCj, values['rhojCj']),
    (E, values['E']),
    (Fj, values['Fj']),
    (rhoCp, values['rhoCp']),
    (Vj, values['Vj']),
    (Tf, values['Tf']),
    (R, values['R'])
])
f

Matrix([
[                   -C_a - 36000000.0*C_a*exp(-6039.25515853045/T) + 10],
[234000000000.0*C_a*exp(-6039.25515853045/T) - 650*T + 150*T_j + 149000],
[                                         -150*T - 600.0*T_j + 223500.0]])

In [50]:
J = J.subs([
    (F, values['F']),
    (C_Af, values['CAf']),
    (V, values['V']),
    (UA, values['UA']),
    (k0, values['k0']),
    (Tj0, values['Tj0']),
    (neg_delH, values['-delH']),
    (rhojCj, values['rhojCj']),
    (E, values['E']),
    (Fj, values['Fj']),
    (rhoCp, values['rhoCp']),
    (Vj, values['Vj']),
    (Tf, values['Tf']),
    (R, values['R'])
])
J

Matrix([
[-1 - 36000000.0*exp(-6039.25515853045/T),          -217413185707.096*C_a*exp(-6039.25515853045/T)/T**2,      0],
[ 234000000000.0*exp(-6039.25515853045/T), 1.41318570709612e+15*C_a*exp(-6039.25515853045/T)/T**2 - 650,    150],
[                                       0,                                                         -150, -600.0]])

In [75]:
#Define function for multivariate Newton-Raphson which stops when tolerance limit is reached and has no maximum number of iterations and keeps count of number of iterations
def newton_raphson(f, J, x0, tol = 1e-6, max_iter = 1000):
    decimal_places = int(-sp.log(tol, 10).evalf()+1)
    x = x0
    iter = 0
    while iter<max_iter:
        iter += 1
        x_new = (x - (J.inv()*f).subs([(C_a, x[0]), (T, x[1]), (Tj, x[2])]))
        #rounding off to the number of decimal places in tolerance
        x_new = sp.Matrix([round(x_new[0].evalf(), decimal_places), round(x_new[1].evalf(), decimal_places), round(x_new[2].evalf(), decimal_places)])
        # x_new = [round(x_new[0].evalf(), decimal_places), round(x_new[1].evalf(), decimal_places), round(x_new[2].evalf(), decimal_places)]
        if (abs(x_new[0]-x[0])+abs(x_new[1]-x[1])+abs(x_new[2]-x[2])) < tol:
            break
        x = x_new
    return x_new

In [77]:
initial_guess = sp.Matrix([0.5, 300, 300])
tol = 1e-6
start = time.time()
print(newton_raphson(f, J, initial_guess, tol))
end = time.time()
print("Time taken:", end - start)

Matrix([[9.0845800], [306.6548799], [295.8362800]])
Time taken: 5.302950143814087


In [80]:
initial_guess = sp.Matrix([9, 200, 300])
tol = 1e-6
start = time.time()
print(newton_raphson(f, J, initial_guess, tol))
end = time.time()
print("Time taken:", end - start)

Matrix([[9.0845800], [306.6548799], [295.8362800]])
Time taken: 6.310132026672363


In [79]:
initial_guess = sp.Matrix([0.5, 300, 200])
tol = 1e-6
start = time.time()
print(newton_raphson(f, J, initial_guess, tol))
end = time.time()
print("Time taken:", end - start)

Matrix([[9.0845800], [306.6548799], [295.8362800]])
Time taken: 5.254057168960571


In [ ]:
+